In [2]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor

In [4]:
## Data Prep

dataset = MNIST(
    root="data",
    train=True,
    transform=Compose([ToTensor(), Normalize(0.5, 0.5)]),
    download=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [16]:
timesteps = np.linspace(80, 0, num_steps) # {80 ...., 0} for 100 elements

# SDE dictates smaller step sizes at low noise levels
sigma_max = 80
sigma_min = 0.002
rho = 7 # raise all elements to 7 while maintaining scale
steps = torch.arange(0, num_steps)
timesteps = ((sigma_max ** (1/rho)) + (steps / (num_steps - 1)) * (sigma_min ** (1/rho) - sigma_max ** (1/rho))) ** rho #scale from 0, num_steps to 80, 0 following some root of step shape—looks like pixel value/time graph
print(timesteps)

tensor([8.0000e+01, 7.5691e+01, 7.1583e+01, 6.7667e+01, 6.3936e+01, 6.0384e+01,
        5.7001e+01, 5.3783e+01, 5.0721e+01, 4.7810e+01, 4.5043e+01, 4.2415e+01,
        3.9920e+01, 3.7551e+01, 3.5304e+01, 3.3173e+01, 3.1153e+01, 2.9240e+01,
        2.7428e+01, 2.5713e+01, 2.4091e+01, 2.2558e+01, 2.1109e+01, 1.9740e+01,
        1.8448e+01, 1.7230e+01, 1.6081e+01, 1.4998e+01, 1.3979e+01, 1.3019e+01,
        1.2117e+01, 1.1268e+01, 1.0471e+01, 9.7232e+00, 9.0214e+00, 8.3634e+00,
        7.7470e+00, 7.1700e+00, 6.6302e+00, 6.1256e+00, 5.6543e+00, 5.2144e+00,
        4.8042e+00, 4.4220e+00, 4.0661e+00, 3.7351e+00, 3.4274e+00, 3.1418e+00,
        2.8768e+00, 2.6311e+00, 2.4037e+00, 2.1934e+00, 1.9990e+00, 1.8196e+00,
        1.6541e+00, 1.5017e+00, 1.3616e+00, 1.2328e+00, 1.1145e+00, 1.0062e+00,
        9.0695e-01, 8.1625e-01, 7.3344e-01, 6.5793e-01, 5.8919e-01, 5.2670e-01,
        4.6998e-01, 4.1858e-01, 3.7207e-01, 3.3006e-01, 2.9219e-01, 2.5810e-01,
        2.2748e-01, 2.0003e-01, 1.7547e-

In [8]:
#ROUGH FORWARD PASS

num_steps = 100
img_shape = 28 * 28

x = torch.randn(img_shape) * timesteps[0]

for t_curr, t_next in zip(timesteps[:-1], timesteps[1:]):
    
    noise_level = t_next / t_curr
    
    x = x * noise_level + denoise(x, t_curr)
    

ValueError: operands could not be broadcast together with shapes (99,) (100,) 

In [ ]:
#BACKWARD PASS
for real_img in data:
    sigma = np.uniform(0, 80) # noise level
    
    noised_image = real_img + sigma * torch.randn_like(real_img)
    
    denoise_image = denoise(noisy_image, sigma) # must give it a noise level as input so it knows how many times to denoise, etc.
    
    loss = mse(denoise_image, noise_image)
    
    #train denoiser

# Issues w above
noise_image is now scaled to a potentially large value—can be tricky to train and generally unoptimal to have varying input image sizes
scale down

In [ ]:
data_var = 0.5

def denoise(noisy_img, sigma):
    noisy_img_var = sigma**2 + data_var**2
    scaled_noisy_img = noisy_img / noisy_img_var ** 0.5
    return net(scaled_noisy_img, sigma) #optionally, scale sigma to -1 to 1 via log fn

# predicting image vs noise

most models forward pass involve predicting noise given a noisy_img and a sigma level—clean img reconstructed by noisy_img - noise

dynamically adjust how much of the noisy_img is recycled during training (i.e. c_skip should increase in training) and modify return value:

In [ ]:
def denoise(noisy_img, sigma):
    sample_img_var = sigma**2 + data_var**2
    scaled_noisy_img = noisy_img / sample_img_var**2
    return c_skip * noisy_img + c_out * net(scaled_noisy_img, sigma)

# varying loss update sizes
due to c_skip, c_out, and other denoising reasons—the gradient updates has varying magnitude based on the current noise level—adjust loss weight some how!
# allocating training effort
weight could potentially be taken advantage of to train the network specifically on certain noise levels—dedicating network capacity where it ocunts. however this can be performed without scaling the magnitude of the losses simply by sampling noises by sampling noise levels during training with a particular probability disttribution that allocates where needed

In [3]:
# final soln
sigma_data = 0.5 # from transform
P_mean = -1.2       # average noise level (logarithmic)
P_std = 1.2     # spread of random noise levels

def net(scaled_img, scaled_noise):
    some_noise = torch.randn_like(scaled_img)
    return some_noise # the net outputs noise! subtract noise from the dirty_image to get cleaner!

def denoise(noisy_image, sigma):
        # Input, output and skip scale
        c_in = 1 / torch.sqrt(sigma_data**2 + sigma**2)
        c_out = sigma * sigma_data / torch.sqrt(sigma**2 + sigma_data**2)
        c_skip = sigma_data**2 / (sigma**2 + sigma_data**2)
        c_noise = torch.log(sigma) / 4      # noise label warp
 
        # mix the input and network output to extract the clean image
        return c_skip * noisy_image + \
                   c_out  * net(c_in * noisy_image, c_noise)
                   
# training
for clean_image in range(1):
    sigma = np.exp(P_mean + P_std * torch.randn([]))
    
    noisy_image = clean_image + sigma * torch.randn_like(clean_image)
    
    denoised_image = denoise(noisy_image)
    
    weight = (sigma**2 + sigma_data**2) / (sigma * sigma_data)**2
    loss = weight * (denoised_image - clean_image).square().sum()

tensor(0.9747)
